In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
clim = pd.read_csv('data/Indonesia_Climate.csv')

In [4]:
clim.head(3)

,Unnamed: 0,Area,Element,Year,Value
0,0,Indonesia,Temperature change,1990,0.120
1,1,Indonesia,Temperature change,1991,0.203
2,2,Indonesia,Temperature change,1992,0.264


In [5]:
clim.drop(clim.columns[[0, 1]], axis=1, inplace=True)

In [8]:
clim.to_csv('data/Ind_clim.csv', index=False)

In [ ]:
df.to_csv("c:/tmp/courses.csv",header=False, index=False)